In [1]:
import sys, os
import os.path as osp
import numpy as np
import pdb
import ipyvolume as ipv


from latent_3d_points.src.analyze import get_nn_between_codes, get_nn_distance, get_average_distance, get_nn_chamfer, get_nn_chamfer_own, get_avg_chamfer_own, get_chamfer_permut
from latent_3d_points.src.general_utils import apply_augmentations
from latent_3d_points.external.structural_losses.tf_nndistance import nn_distance
from latent_3d_points.src.load_ae_model import load as load_ae, get_feed_data, unpickle_pc_file
from latent_3d_points.src.in_out import create_dir

/home/yz6/.virtualenvs/gan_env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model_path = '/home/yz6/code/latent_3d_points/data/train_single_class_ae_plane_chamfer_nonrotate_2048p/models.ckpt-900' # model path of ae model
train_pc_file = '/home/yz6/code/latent_3d_points/data/train_single_class_ae_plane_chamfer_nonrotate_2048p/train_pc.pkl' # .pkl file of point cloud
test_pc_file = '/home/yz6/code/latent_3d_points/data/test_single_class_ae_plane_chamfer_nonrotate_2048p/test_pc.pkl' # .pkl file of point cloud
z_rotate = 'False' # 'True' or 'False'
num_points = 2048
generated_hidden_file = '../data/lgan_single_class_ae_plane_chamfer_nonrotate_2048p/hidden_generated.npy' # gerenated .npy file by gan

In [3]:
ae, conf = load_ae(model_path, z_rotate, num_points)

train_pc_data = unpickle_pc_file(train_pc_file)
train_feed = get_feed_data(train_pc_data, conf)

batchsize = 100
n_examples = train_pc_data.num_examples
n_batch = n_examples/batchsize
hidden_result = list()
reconst_result = list()
for _ in xrange(n_batch):
    feed_pc, feed_model_names, _ = train_pc_data.next_batch(batchsize)
    if z_rotate == 'True':
        feed_pc = apply_augmentations(feed_pc, conf)
    hidden = ae.transform(feed_pc)
    reconst, _ = ae.reconstruct(feed_pc)
    hidden_result.append(hidden)
    reconst_result.append(reconst)
train_hidden = np.concatenate(hidden_result, axis=0)
train_reconst = np.concatenate(reconst_result, axis=0)



test_pc_data = unpickle_pc_file(test_pc_file)
test_feed = get_feed_data(test_pc_data, conf)

batchsize = 100
n_examples = test_pc_data.num_examples
n_batch = n_examples/batchsize
hidden_result = list()
reconst_result = list()
for _ in xrange(n_batch):
    feed_pc, feed_model_names, _ = test_pc_data.next_batch(batchsize)
    if z_rotate == 'True':
        feed_pc = apply_augmentations(feed_pc, conf)
    hidden = ae.transform(feed_pc)
    reconst, _ = ae.reconstruct(feed_pc)
    hidden_result.append(hidden)
    reconst_result.append(reconst)
test_hidden = np.concatenate(hidden_result, axis=0)
test_reconst = np.concatenate(reconst_result, axis=0)


generated_hidden= np.load(generated_hidden_file)
generated_reconstr = ae.decode(generated_hidden)

number of points sampled per object: 2048
Building Encoder
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("single_class_ae_plane_chamfer_nonrotate_2048p_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("single_class_ae_plane_chamfer_nonrotate_2048p_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("single_class_ae_plane_chamfer_nonrotate_2048p_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024 bnorm params =  512
Tensor("single_class_ae_plane_chamfer_nonrotate_2048p_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896 bnorm params =  256
Tensor(

In [4]:
print("nn avg distance between train and generate")
train_code2gen, train_gen2code, train_dist_list, train_mean_nn_dist = get_nn_between_codes(train_hidden, generated_hidden)

# print("train, generated, gen2code")
# print(train_gen2code)



print("nn avg distance between test and generate")
test_code2gen, test_gen2code, test_dist_list, test_mean_nn_dist = get_nn_between_codes(test_hidden, generated_hidden)

nn avg distance between train and generate
nn avg distance between test and generate


In [5]:


print("train-gen fidelity")
print(train_mean_nn_dist)

print("test-gen fidelity")
print(test_mean_nn_dist)


print("train-gen coverage")
print(len(train_code2gen.keys())*1.0/ len(train_hidden))

print("test-gen coverage")
print(len(test_code2gen.keys())*1.0/ len(test_hidden))


train-gen fidelity
0.843449
test-gen fidelity
0.87590486
train-gen coverage
0.0971875
test-gen coverage
0.285


In [122]:
i=60

In [127]:
train_code2gen[1495]

[(6912, 0.9027868)]

In [121]:
# sorted(train_code2gen.iteritems(),key=lambda (k,v): len(v),reverse=True)
counter = [(k, len(v)) for k, v in train_code2gen.iteritems()]
sorted(counter, key=lambda(k, v): v, reverse=True)

[(3009, 563),
 (2777, 515),
 (1933, 502),
 (672, 499),
 (2838, 436),
 (500, 409),
 (1700, 369),
 (1402, 330),
 (2030, 247),
 (1883, 233),
 (1303, 230),
 (1562, 229),
 (1391, 206),
 (1216, 190),
 (1703, 184),
 (2623, 158),
 (1869, 144),
 (3048, 144),
 (577, 112),
 (422, 100),
 (824, 92),
 (3024, 91),
 (739, 90),
 (1353, 90),
 (2332, 86),
 (2221, 85),
 (2847, 84),
 (3193, 79),
 (3189, 75),
 (184, 75),
 (762, 75),
 (2067, 71),
 (2987, 71),
 (350, 64),
 (1794, 62),
 (380, 62),
 (1054, 61),
 (565, 61),
 (1331, 61),
 (3016, 61),
 (2907, 59),
 (2027, 59),
 (254, 58),
 (635, 54),
 (1694, 54),
 (767, 54),
 (2860, 51),
 (1840, 50),
 (1867, 46),
 (553, 45),
 (601, 45),
 (1136, 45),
 (2110, 44),
 (2487, 43),
 (564, 40),
 (3159, 40),
 (217, 37),
 (87, 35),
 (2132, 34),
 (247, 34),
 (2786, 31),
 (1424, 31),
 (948, 31),
 (724, 30),
 (800, 29),
 (1062, 28),
 (2893, 28),
 (840, 27),
 (1382, 27),
 (949, 27),
 (2178, 26),
 (1509, 26),
 (1721, 26),
 (1713, 25),
 (1040, 24),
 (356, 24),
 (2449, 24),
 (3081

In [128]:
i=6912
obj = generated_reconstr[i]
obj = obj * 2
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x, y, z, size=0.5, marker="sphere", data_max = 0.5)

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='|S5'), geo=u'sphere', size=array(0.5), size_selected=array(2.6), x=array([-0.65202856,  0.06935644,  0.09544323, ...,  0.6883589 ,
       -0.3684781 ,  0.2784949 ], dtype=float32), y=array([0.00646324, 0.00612869, 0.26938236, ..., 0.02103635, 0.01277317,
       0.029916  ], dtype=float32), z=array([-0.00552683, -0.00253215, -0.11888531, ..., -0.12625279,
       -0.00800516, -0.15708065], dtype=float32))], style={'box': {'visible': True}, 'axes': {'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=400, xlim=[-0.7198435664176941, 1.0], ylim=[-0.6404635906219482, 1.0], zlim=[-0.20361343026161194, 1.0]),))

In [39]:
obj = train_feed[train_gen2code[i][0]]
obj = obj * 2
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x, y, z, size=0.5, marker="sphere", data_max = 0.5)

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='|S5'), geo=u'sphere', size=array(0.5), size_selected=array(2.6), x=array([-0.15582812, -0.05929401,  0.5506653 , ..., -0.15843976,
       -0.4389303 , -0.56726325], dtype=float32), y=array([ 0.40197614,  0.04440415, -0.04832111, ...,  0.60351205,
        0.00620614, -0.02543628], dtype=float32), z=array([-0.09519428, -0.02459428, -0.06027044, ..., -0.06363811,
        0.0458362 , -0.07350007], dtype=float32))], style={'box': {'visible': True}, 'axes': {'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=400, xlim=[-0.6527520418167114, 1.0], ylim=[-0.7219737768173218, 1.0], zlim=[-0.20649409294128418, 1.0]),))

In [9]:
obj = test_feed[test_gen2code[i][0]]
obj = obj * 2
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x, y, z, size=0.5, marker="sphere", data_max = 0.5)

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='|S5'), geo=u'sphere', size=array(0.5), size_selected=array(2.6), x=array([-0.01321713, -0.5727943 ,  0.5702367 , ..., -0.70980763,
       -0.5540135 , -0.68838644], dtype=float32), y=array([-0.06490679,  0.03369065, -0.04954353, ...,  0.19984129,
       -0.01071527, -0.2222646 ], dtype=float32), z=array([-0.05039015, -0.06158118, -0.11403786, ..., -0.0378079 ,
        0.01288616, -0.0361436 ], dtype=float32))], style={'box': {'visible': True}, 'axes': {'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=400, xlim=[-0.714247465133667, 1.0], ylim=[-0.6653817892074585, 1.0], zlim=[-0.18602614104747772, 1.0]),))

In [10]:
obj = train_reconstr[train_gen2code[i][0]]
obj = obj * 2
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x, y, z, size=.5, marker="sphere", data_max = 0.5)


NameError: name 'train_reconstr' is not defined